In [1]:
# Wczytanie bibliotek

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import io
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import keras
import pandas as pd
from numpy import savetxt

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%load_ext tensorboard
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

In [2]:
# Wczytanie folderu

images_dir = "C:/wikiart/"

In [3]:
# Wczytanie pliku z klasami

training_set = pd.read_csv('C:/wikiart/wclasses.csv')
training_set

,file,artist,genre,style
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,22,133,161
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,20,136,144
2,Post_Impressionism/paul-cezanne_portrait-of-th...,16,135,160
3,Impressionism/pierre-auguste-renoir_young-girl...,17,131,152
4,Romanticism/ivan-aivazovsky_morning-1851.jpg,9,139,163
...,...,...,...,...
81439,Naive_Art_Primitivism/mary-fedden_butterflies-...,0,139,155
81440,Naive_Art_Primitivism/arman-manookian_watercol...,0,139,155
81441,Naive_Art_Primitivism/andre-bauchant_exotic-fl...,0,139,155
81442,Naive_Art_Primitivism/ivan-generalic_cows-in-a...,0,139,155


In [4]:
# Okreslenie wieku powstania na bazie nazw plików, dodanie do kolejnej kolumny

training_set['century']=training_set['artist']

for i in training_set.index:
  filename = os.path.basename(training_set['file'][i])
  parts = filename.split('-')
  yearonly = parts[len(parts)-1].split('.')[0]
  try:
    yearonly=int(yearonly) 
  except ValueError:
    yearonly=None

  try:
    if 1900<yearonly<2001:
      training_set['century'][i]=20
    elif 1800<yearonly<1901:
      training_set['century'][i]=19
    elif 1700<yearonly<1801:
      training_set['century'][i]=18
    elif 1600<yearonly<1701:
      training_set['century'][i]=17
    elif 1500<yearonly<1601:
      training_set['century'][i]=16
    elif 1400<yearonly<1501:
      training_set['century'][i]=15
    else:
      training_set['century'][i]=-1
  except TypeError:
    training_set['century'][i]=-1

<ipython-input-4-ad0fe026b85b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_set['century'][i]=19
<ipython-input-4-ad0fe026b85b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_set['century'][i]=17
<ipython-input-4-ad0fe026b85b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_set['century'][i]=-1
<ipython-input-4-ad0fe026b85b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [5]:
# Złożenie training_set, odrzucenie kategorii "Nieznany Gatunek"

training_imgs = list(training_set['file'])
training_labels_1  = list(training_set['artist'])
training_labels_2 = list(training_set['genre'])
training_labels_3 = list(training_set['style'])
training_labels_4 = list(training_set['century'])

training_set = pd.DataFrame( {'Painting': training_imgs, 'Artist' : training_labels_1, 'Genre' : training_labels_2, 'Style' : training_labels_3, 'Century' : training_labels_4})
training_set

,Painting,Artist,Genre,Style,Century
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,22,133,161,19
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,20,136,144,17
2,Post_Impressionism/paul-cezanne_portrait-of-th...,16,135,160,-1
3,Impressionism/pierre-auguste-renoir_young-girl...,17,131,152,20
4,Romanticism/ivan-aivazovsky_morning-1851.jpg,9,139,163,19
...,...,...,...,...,...
81439,Naive_Art_Primitivism/mary-fedden_butterflies-...,0,139,155,20
81440,Naive_Art_Primitivism/arman-manookian_watercol...,0,139,155,-1
81441,Naive_Art_Primitivism/andre-bauchant_exotic-fl...,0,139,155,20
81442,Naive_Art_Primitivism/ivan-generalic_cows-in-a...,0,139,155,20


In [6]:
training_set

,Painting,Artist,Genre,Style,Century
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,22,133,161,19
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,20,136,144,17
2,Post_Impressionism/paul-cezanne_portrait-of-th...,16,135,160,-1
3,Impressionism/pierre-auguste-renoir_young-girl...,17,131,152,20
4,Romanticism/ivan-aivazovsky_morning-1851.jpg,9,139,163,19
...,...,...,...,...,...
81439,Naive_Art_Primitivism/mary-fedden_butterflies-...,0,139,155,20
81440,Naive_Art_Primitivism/arman-manookian_watercol...,0,139,155,-1
81441,Naive_Art_Primitivism/andre-bauchant_exotic-fl...,0,139,155,20
81442,Naive_Art_Primitivism/ivan-generalic_cows-in-a...,0,139,155,20


In [7]:
# Dodanie nazwisk artystów z nazw plików ze względu na błędne przypisanie niektórych obrazów Nieznanemu Artyście 
# (poprzez oznaczenie "0" - np ostatnie obrazy poniższsego dataframe)

training_set['Artist_painting'] = training_set.Painting.str.split("/",expand=True)[1]
training_set['Artist_name'] = training_set.Artist_painting.str.split("_",expand=True)[0]
training_set

,Painting,Artist,Genre,Style,Century,Artist_painting,Artist_name
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,22,133,161,19,vincent-van-gogh_pine-trees-in-the-fen-1884.jpg,vincent-van-gogh
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,20,136,144,17,rembrandt_the-angel-appearing-to-the-shepherds...,rembrandt
2,Post_Impressionism/paul-cezanne_portrait-of-th...,16,135,160,-1,paul-cezanne_portrait-of-the-artist-s-son.jpg,paul-cezanne
3,Impressionism/pierre-auguste-renoir_young-girl...,17,131,152,20,pierre-auguste-renoir_young-girl-seated-in-a-m...,pierre-auguste-renoir
4,Romanticism/ivan-aivazovsky_morning-1851.jpg,9,139,163,19,ivan-aivazovsky_morning-1851.jpg,ivan-aivazovsky
...,...,...,...,...,...,...,...
81439,Naive_Art_Primitivism/mary-fedden_butterflies-...,0,139,155,20,mary-fedden_butterflies-at-canaux-1973.jpg,mary-fedden
81440,Naive_Art_Primitivism/arman-manookian_watercol...,0,139,155,-1,arman-manookian_watercolor-design-for-mural.jpg,arman-manookian
81441,Naive_Art_Primitivism/andre-bauchant_exotic-fl...,0,139,155,20,andre-bauchant_exotic-flowers-1933.jpg,andre-bauchant
81442,Naive_Art_Primitivism/ivan-generalic_cows-in-a...,0,139,155,20,ivan-generalic_cows-in-a-landscape-1957.jpg,ivan-generalic


In [8]:
# Odrzucenie obrazów zbyt mało płodnych artystów, powodujących błedy, nieznanych gatunków i o nieznanym roku powstania 
# lub nie zawierającym się w przedziale od XV do XX wieku

training_set_count = training_set.groupby(['Artist_name']).size().reset_index(name='count')
training_set_count = training_set_count[training_set_count['count'] > 250]
training_set_count = training_set_count[training_set_count['count'] != 350]
training_set =  training_set[training_set['Artist_name'].isin(training_set_count['Artist_name'])]
training_set = training_set[training_set.Genre !=139]
training_set = training_set[training_set.Century != -1]
training_set

,Painting,Artist,Genre,Style,Century,Artist_painting,Artist_name
0,Realism/vincent-van-gogh_pine-trees-in-the-fen...,22,133,161,19,vincent-van-gogh_pine-trees-in-the-fen-1884.jpg,vincent-van-gogh
1,Baroque/rembrandt_the-angel-appearing-to-the-s...,20,136,144,17,rembrandt_the-angel-appearing-to-the-shepherds...,rembrandt
3,Impressionism/pierre-auguste-renoir_young-girl...,17,131,152,20,pierre-auguste-renoir_young-girl-seated-in-a-m...,pierre-auguste-renoir
5,Art_Nouveau_Modern/boris-kustodiev_meeting-eas...,1,131,143,20,boris-kustodiev_meeting-easter-day-1917.jpg,boris-kustodiev
7,Romanticism/ivan-aivazovsky_night-in-the-ukrai...,9,133,163,19,ivan-aivazovsky_night-in-the-ukraine-1871.jpg,ivan-aivazovsky
...,...,...,...,...,...,...,...
67756,Abstract_Expressionism/sam-francis_variant-iii...,27,129,140,20,sam-francis_variant-iii-from-bright-jade-ghost...,sam-francis
67770,Color_Field_Painting/sam-francis_untitled-sf78...,27,129,145,20,sam-francis_untitled-sf78-235-1978.jpg,sam-francis
67773,Abstract_Expressionism/sam-francis_2-pri-rain-...,27,129,140,20,sam-francis_2-pri-rain-1964.jpg,sam-francis
67777,Abstract_Expressionism/sam-francis_sketch-1960...,27,129,140,20,sam-francis_sketch-1960.jpg,sam-francis


In [9]:
# Zmiana typu na str

training_set.Style = training_set.Style.astype(str)
training_set.Genre = training_set.Genre.astype(str)
training_set.Century = training_set.Century.astype(str)

In [10]:
# Nazwy klas (z przetłumaczeniem)

columns_style = ['Ekspresjonizm Abstrakcyjny', 'Taszyzm', 'Kubizm Analityczny', 'Art Noveau', 'Barok', 'Malarstwo Barwnych Płaszczyzn', 'Realizm Współczesny', 'Kubizm', 'Wczesny Renesans', 'Ekspresjonizm', 'Fowizm', 'Wysoki Renesans', 'Impresjonizm', 'Późny Renesans', 'Minimalizm', 'Prymitywizm', 'Nowy Realizm', 'Północny Renesans', 'Puentylizm', 'Pop Art', 'Postimpresjonizm', 'Realizm', 'Rokoko', 'Romantyzm', 'Symbolizm', 'Kubizm Syntetyczny', 'Ukiyo-e']
columns_genre = ['Abstrakcja', 'Pejzaż Miejski', 'Malarstwo Rodzajowe', 'Ilustracja', 'Krajobraz', 'Akt', 'Portret', 'Malarstwo Religijne', 'Szkic', 'Martwa Natura']
columns_century = ['XV', 'XVI', 'XVII', 'XVIII', 'XIX', 'XX']
columns_artist = training_set_count['Artist_name'].tolist()

In [11]:
# Wczytanie modeli

model_Artist = keras.models.load_model("C:/Users/jarek/ARTIST62_best_model.hdf5")
model_Style = keras.models.load_model("C:/Users/jarek/STYLE32_best_model.hdf5")
model_Genre = keras.models.load_model("C:/Users/jarek/GENRE72_best_model.hdf5")
model_Century = keras.models.load_model("C:/wikiart/Model_CENTURY4.hpf5")

In [12]:
# Dokonanie przewidywań dla wszystkich obrazów we wszystkich kategoriach; zapisanie do .csv

train_input_shape = (128, 128, 3)

artist_generator = ImageDataGenerator().flow_from_dataframe(
                                        dataframe = training_set,
                                        directory=images_dir,
                                        x_col="Painting",
                                        y_col="Artist_name",
                                        class_mode="categorical",
                                        shuffle=False,
                                        target_size=train_input_shape[0:2],
                                        batch_size=1)

artist_predict = model_Artist.predict(artist_generator)

data = artist_predict
savetxt('C:/users/jarek/artist_predict_mini.csv', data, delimiter=',')

style_generator = ImageDataGenerator().flow_from_dataframe(
                                        dataframe = training_set,
                                        directory=images_dir,
                                        x_col="Painting",
                                        y_col="Style",
                                        class_mode="categorical",
                                        shuffle=False,
                                        target_size=train_input_shape[0:2],
                                        batch_size=1)    
    
style_predict = model_Style.predict(style_generator)

data = style_predict
savetxt('C:/users/jarek/style_predict_mini.csv', data, delimiter=',')
    
genre_generator = ImageDataGenerator().flow_from_dataframe(
                                        dataframe = training_set,
                                        directory=images_dir,
                                        x_col="Painting",
                                        y_col="Genre",
                                        shuffle=False,       
                                        class_mode="categorical",
                                        target_size=train_input_shape[0:2],
                                        batch_size=1)    
        
genre_predict = model_Genre.predict(genre_generator)

data = genre_predict
savetxt('C:/users/jarek/genre_predict_mini.csv', data, delimiter=',')

century_generator = ImageDataGenerator(rescale = 1./255).flow_from_dataframe(
                                        dataframe = training_set,
                                        directory=images_dir,
                                        x_col="Painting",
                                        y_col="Century",
                                        class_mode="categorical",
                                        shuffle=False,
                                        target_size=train_input_shape[0:2],
                                        batch_size=1)  
    
century_predict = model_Century.predict(century_generator)

data = century_predict
savetxt('C:/users/jarek/century_predict_mini.csv', data, delimiter=',')

Found 14498 validated image filenames belonging to 56 classes.
Found 14498 validated image filenames belonging to 21 classes.
Found 14498 validated image filenames belonging to 10 classes.
Found 14498 validated image filenames belonging to 6 classes.


In [13]:
### Przerwa

In [14]:
# Wczytanie przewidywań

prediction_style = pd.read_csv('C:/users/jarek/style_predict_mini.csv', names=columns_style, header=None)
prediction_genre = pd.read_csv('C:/users/jarek/genre_predict_mini.csv', names=columns_genre, header=None)
prediction_century = pd.read_csv('C:/users/jarek/century_predict_mini.csv', names=columns_century, header=None)
prediction_artist = pd.read_csv('C:/users/jarek/artist_predict_mini.csv', names=columns_artist, header=None)

In [15]:
# Złożenie przwidywań

prediction_set = pd.concat([prediction_style, prediction_genre,prediction_artist,prediction_century ], axis=1)

In [16]:
training_set.reset_index(drop=True,inplace=True)

In [17]:
# Zestaw przewidywań dla każdej kategorii

prediction_set_artist = pd.concat([prediction_set, training_set['Artist_name']], axis=1)
prediction_set_artist.to_csv('C:/wikiart/prediction_set_artist.csv')

prediction_set_genre = pd.concat([prediction_set, training_set['Genre']], axis=1)
prediction_set_genre.to_csv('C:/wikiart/prediction_set_genre.csv')

prediction_set_style = pd.concat([prediction_set, training_set['Style']], axis=1)
prediction_set_style.to_csv('C:/wikiart/prediction_set_style.csv')

prediction_set_century = pd.concat([prediction_set, training_set['Century']], axis=1)
prediction_set_century.to_csv('C:/wikiart/prediction_set_century.csv')